# Limpieza

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sidetable # si nos da un error este comando es por no tenerlo instalado
#!pip install holidays
import holidays
from datetime import date, datetime

pd.options.display.max_columns=None
# establecer el tamaño de nuestras figuras
plt.rcParams["figure.figsize"] = (10,8)

In [3]:
df = pd.read_csv('data/bikes.csv', index_col = 0)

In [4]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 97.0+ KB


In [6]:
df.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [7]:
df.duplicated().sum()

0

### Renombramos las columnas

Nuestro df contiene las siguientes columnas:
- registro = (instant), es el índice
- fecha = (dteday)
- estacion = (season) Hay que cambiarla, algunas estaciones no coinciden con la fecha.
- año = (year) También hay que cambiarla, está en 0  y 1 (2018,2019)
- mes = (month) 
- festivo = (holiday) 0: laborales, 1:festivos, creemos que tomará festivos.
- dia_semana = (weekday) 
- no_laboral = (workingday) Cuenta como 0 el laboral y el 1 el no laboral
- clima = ('weathersit')
- 'temperatura' =  (temp)
- 'sens_termica'=  ('atemp')
- humedad = (hum)
- viento = (windspeed)
- ocasionales = (casual)
- registrados = (registred)
- total = (cnt)


In [8]:
df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [9]:
dic_columnas = {'instant': 'registro', 'dteday': 'fecha', 'season': 'estacion', 'yr': 'año', 'mnth':'mes', 'holiday':'festivo', 'weekday':'dia_semana',
       'workingday':'no_laboral', 'weathersit':'clima', 'temp':'temperatura', 'atemp':'sens_termica', 'hum':'humedad', 'windspeed':'viento', 
       'casual':'ocasionales', 'registered':'registrados', 'cnt':'total'}

In [10]:
df.rename(columns = dic_columnas, inplace = True)

In [11]:
df.head(2)

,registro,fecha,estacion,año,mes,festivo,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801


In [12]:
df['festivo'].value_counts()

0    709
1     21
Name: festivo, dtype: int64

In [13]:
festivos = df[df['festivo'] == 1]
festivos.head(21)

,registro,fecha,estacion,año,mes,festivo,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total
16,17,17-01-2018,spring,0,1,1,1,0,2,7.209153,8.83855,53.7500,12.999139,117,883,1000
51,52,21-02-2018,spring,0,2,1,1,0,2,12.436653,14.20375,60.5000,20.625682,195,912,1107
104,105,15-04-2018,summer,0,4,1,5,0,1,18.313347,22.09565,67.1250,15.167125,642,2484,3126
149,150,30-05-2018,summer,0,5,1,1,0,1,30.066653,33.55460,68.5000,8.792075,1549,2549,4098
184,185,04-07-2018,autumn,0,7,1,1,0,2,29.793347,33.27085,63.7917,5.459106,3065,2978,6043
247,248,05-09-2018,autumn,0,9,1,1,0,2,27.606653,31.28230,79.0417,14.250632,1236,2115,3351
282,283,10-10-2018,winter,0,10,1,1,0,1,23.404153,27.14625,73.3750,2.834381,1514,3603,5117
314,315,11-11-2018,winter,0,11,1,5,0,1,13.290847,15.34085,44.6250,21.083225,440,2928,3368
327,328,24-11-2018,winter,0,11,1,4,0,1,15.306653,18.62355,54.9167,11.209368,560,935,1495
359,360,26-12-2018,spring,0,12,1,1,0,1,13.191299,15.77675,50.6957,16.044155,430,887,1317


#### Cambiamos la columna año, mes y día_semana

In [14]:
df['dia_semana'].unique()

array([6, 0, 1, 2, 3, 4, 5])

Cambiamos el formato a la columna fecha

In [15]:
df['fecha_nueva'] = pd.to_datetime(df["fecha"], format="%d-%m-%Y")

In [16]:
df['festividad'] = pd.Series(df.fecha_nueva).apply(lambda x: holidays.CountryHoliday('US').get(x)).values

In [17]:
df['festividad_1'] = df['festividad'].astype('bool').astype('int')

In [18]:
df['festividad_1'].value_counts()

0    709
1     21
Name: festividad_1, dtype: int64

In [19]:
df.head(2)

,registro,fecha,estacion,año,mes,festivo,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2018-01-02,None,0


Cambiamos las estaciones

Primero renombramos:

In [20]:
#df['año_bueno'] = df['fecha_nueva'].apply(datetime.date.today().year)

df['año_bueno'] = pd.DatetimeIndex(df['fecha_nueva']).year 

In [21]:
df['año_bueno'].value_counts()

2018    365
2019    365
Name: año_bueno, dtype: int64

In [22]:
df['año_bueno'].unique()

array([2018, 2019])

In [23]:
year = df['año_bueno'].unique()

for x in year:
    seasons = [('winter', (date(x,  1,  1),  date(x,  3, 20))),
            ('spring', (date(x,  3, 21),  date(x,  6, 20))),
            ('summer', (date(x,  6, 21),  date(x,  9, 22))),
            ('autumn', (date(x,  9, 23),  date(x, 12, 20))),
            ('winter', (date(x, 12, 21),  date(x, 12, 31)))]
print(seasons)


    

#print(get_season(date.today()))

[('winter', (datetime.date(2019, 1, 1), datetime.date(2019, 3, 20))), ('spring', (datetime.date(2019, 3, 21), datetime.date(2019, 6, 20))), ('summer', (datetime.date(2019, 6, 21), datetime.date(2019, 9, 22))), ('autumn', (datetime.date(2019, 9, 23), datetime.date(2019, 12, 20))), ('winter', (datetime.date(2019, 12, 21), datetime.date(2019, 12, 31)))]


In [24]:

def sacar_estacion(fecha):
    if fecha.month == 2 or fecha.month == 1:
        return "invierno"
    elif fecha.month == 3 and fecha.day <= 20:
        return "invierno"
    elif fecha.month == 12 and fecha.day >20:
        return "invierno"
    elif fecha.month == 4 or fecha.month == 5:
        return "primavera"
    elif fecha.month == 3 and fecha.day >20:
        return "primavera"
    elif fecha.month == 6 and fecha.day <=20:
        return "primavera"
    elif fecha.month == 7 or fecha.month == 8:
        return "verano"
    elif fecha.month == 6 and fecha.day > 20:
        return "verano"
    elif fecha.month == 9 and fecha.day < 23:
        return "verano"
    elif fecha.month == 10 or fecha.month == 11:
        return "otoño"
    elif fecha.month == 9 and fecha.day > 22:
        return "otoño"
    elif fecha.month == 12 and fecha.day <= 20:
        return "otoño"


    
    

In [25]:
df["estacion_correcta"] = df["fecha_nueva"].apply(sacar_estacion)

In [26]:
df.tail(15)

,registro,fecha,estacion,año,mes,festivo,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta
715,716,17-12-2019,winter,1,12,0,1,1,2,16.126653,20.07500,90.7083,6.583306,212,4373,4585,2019-12-17,None,0,2019,otoño
716,717,18-12-2019,winter,1,12,0,2,1,1,16.844153,20.48540,66.6250,14.834068,433,5124,5557,2019-12-18,None,0,2019,otoño
717,718,19-12-2019,winter,1,12,0,3,1,1,13.632500,17.10810,62.5417,12.334164,333,4934,5267,2019-12-19,None,0,2019,otoño
718,719,20-12-2019,winter,1,12,0,4,1,2,13.530000,16.76085,66.7917,8.875021,314,3814,4128,2019-12-20,None,0,2019,otoño
719,720,21-12-2019,spring,1,12,0,5,1,2,13.393347,15.08835,55.6667,25.083661,221,3402,3623,2019-12-21,None,0,2019,invierno
720,721,22-12-2019,spring,1,12,0,6,0,1,10.899153,11.80565,44.1250,27.292182,205,1544,1749,2019-12-22,None,0,2019,invierno
721,722,23-12-2019,spring,1,12,0,0,0,1,10.079153,12.97355,51.5417,8.916561,408,1379,1787,2019-12-23,None,0,2019,invierno
722,723,24-12-2019,spring,1,12,0,1,1,2,9.483464,12.94500,79.1304,5.174437,174,746,920,2019-12-24,None,0,2019,invierno
723,724,25-12-2019,spring,1,12,1,2,0,2,11.943464,14.72325,73.4783,11.304642,440,573,1013,2019-12-25,Christmas Day,1,2019,invierno
724,725,26-12-2019,spring,1,12,0,3,1,3,9.976653,11.01665,82.3333,21.208582,9,432,441,2019-12-26,None,0,2019,invierno


In [27]:
#Creamos un dataframe donde vamos a ir realizando los siguientes cambios.
df2=df.copy()
df2.head(3)

,registro,fecha,estacion,año,mes,festivo,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2018-01-02,None,0,2018,invierno
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,2018-01-03,None,0,2018,invierno


In [28]:
df2.drop(['estacion','festivo'],axis=1,inplace=True)

In [29]:
df2.head(1)

,registro,fecha,año,mes,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta
0,1,01-01-2018,0,1,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno


In [30]:
df2['mes_bueno'] = pd.DatetimeIndex(df2['fecha_nueva']).month
df2.head(1)

,registro,fecha,año,mes,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno
0,1,01-01-2018,0,1,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1


In [31]:
df2.drop(['fecha','año','mes'],axis=1,inplace=True)
df2.head(1)

,registro,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno
0,1,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1


In [32]:
df2['dia_semana_nuevo']=df2['fecha_nueva'].dt.weekday

In [33]:
df2.head(8)

,registro,dia_semana,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo
0,1,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1,0
1,2,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2018-01-02,None,0,2018,invierno,1,1
2,3,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,2018-01-03,None,0,2018,invierno,1,2
3,4,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,2018-01-04,None,0,2018,invierno,1,3
4,5,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,2018-01-05,None,0,2018,invierno,1,4
5,6,4,1,1,8.378268,11.66045,51.8261,6.000868,88,1518,1606,2018-01-06,None,0,2018,invierno,1,5
6,7,5,1,2,8.057402,10.44195,49.8696,11.304642,148,1362,1510,2018-01-07,None,0,2018,invierno,1,6
7,8,6,0,2,6.765000,8.11270,53.5833,17.875868,68,891,959,2018-01-08,None,0,2018,invierno,1,0


In [34]:
df2.drop(['dia_semana'],axis=1,inplace=True)
df2.head(1)

,registro,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo
0,1,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1,0


In [35]:
df2['no_laboral'].value_counts()

1    499
0    231
Name: no_laboral, dtype: int64

In [36]:
#Creamos una funcion para crear una columna con los dias labroales.
def sacar_laboral(df):
    if (df['dia_semana_nuevo']<=4) and (df['festividad_1']== 0):
        return 1
    else:
        return 0

In [37]:
df2['no_laboral_nuevo']=df2.apply(sacar_laboral,axis=1)

In [38]:
df2.head(3)

,registro,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo,no_laboral_nuevo
0,1,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1,0,0
1,2,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2018-01-02,None,0,2018,invierno,1,1,1
2,3,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,2018-01-03,None,0,2018,invierno,1,2,1


In [39]:
df2['no_laboral_nuevo'].value_counts()
#Sí ha habido cambios en el numero de dias labroales detectados.

1    502
0    228
Name: no_laboral_nuevo, dtype: int64

In [40]:
df3=df2.copy()
df3.head(3)

,registro,no_laboral,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo,no_laboral_nuevo
0,1,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1,0,0
1,2,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2018-01-02,None,0,2018,invierno,1,1,1
2,3,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,2018-01-03,None,0,2018,invierno,1,2,1


In [41]:
df3.drop(['no_laboral'],axis=1,inplace=True)
df3.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo,no_laboral_nuevo
0,1,2,14.110847,18.18125,80.5833,10.749882,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1,0,0
1,2,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2018-01-02,None,0,2018,invierno,1,1,1
2,3,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,2018-01-03,None,0,2018,invierno,1,2,1


Limpiamos temperatura, sensación térmica, humedad y viento para dejarlas con 1 decimal.

In [47]:
df3['temperatura'] = df3['temperatura'].round(1)
df3['sens_termica'] = df3['sens_termica'].round(1)
df3['humedad'] = df3['humedad'].round(0)
df3['viento'] = df3['viento'].round(1)

df3.head(2)

,registro,clima,temperatura,sens_termica,humedad,viento,ocasionales,registrados,total,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo,no_laboral_nuevo
0,1,2,14.1,18.2,81.0,10.7,331,654,985,2018-01-01,New Year's Day,1,2018,invierno,1,0,0
1,2,2,14.9,17.7,70.0,16.7,131,670,801,2018-01-02,None,0,2018,invierno,1,1,1


In [48]:
df3.to_csv('data/bikes_limpio.csv')